In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **D**ecision Tree Classifier****

In [ ]:
# Importing the required packages 
import numpy as np 
import pandas as pd 
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from IPython.display import Image
from sklearn import tree
from sklearn.tree import export_graphviz
import graphviz
from sklearn.feature_selection import SelectKBest, f_regression

In [ ]:
# Function importing Dataset 
def importdata(): 
    GDM_data = pd.read_csv('../input/gdm-nonull/GDM_nonull.csv')
    # remove 1h glucose column
#     GDM_data = GDM_data.drop('1h glucose', axis =1)
    return GDM_data

In [ ]:
# Function to split the dataset 
def splitdataset(balance_data): 
  
    # Separating the target variable 
    X = balance_data.drop('Glucose Result', axis =1)

    Y = balance_data['Glucose Result']
  
    # Splitting the dataset into train and test 
    X_train, X_test, Y_train, Y_test = train_test_split(  
    X, Y, test_size = 0.25, random_state = 100) 
      
    return X, Y, X_train, X_test, Y_train, Y_test

In [ ]:
# Function to perform training with giniIndex. 
def train_using_gini(X_train, X_test, Y_train): 
  
    # Creating the classifier object 
    clf_gini = DecisionTreeClassifier(criterion = "gini", 
            random_state = 100,max_depth=10, min_samples_leaf=5) 
  
    # Performing training 
    clf_gini.fit(X_train, Y_train) 
    return clf_gini 

In [ ]:
# Function to perform training with entropy. 
def train_using_entropy(X_train, X_test, Y_train): 
  
    # Decision tree with entropy 
    clf_entropy = DecisionTreeClassifier( 
            criterion = "entropy", random_state = 100, 
            max_depth = 10, min_samples_leaf = 5) 
  
    # Performing training 
    clf_entropy.fit(X_train, Y_train) 
    return clf_entropy 

In [ ]:
# Function to make predictions 
def prediction(X_test, clf_object): 
  
    # Predicton on test with giniIndex 
    Y_pred = clf_object.predict(X_test) 
    print("Predicted values:") 
    print(Y_pred) 
    return Y_pred  

In [ ]:
# Function to calculate accuracy 
def cal_accuracy(y_test, Y_pred): 
              
    print ("Accuracy : ", 
    accuracy_score(y_test,Y_pred)*100) 
      

In [ ]:
# Building Phase 
data = importdata() 
data = data.sample(frac = 1) 

X, Y, X_train, X_test, Y_train, Y_test = splitdataset(data) 
clf_gini = train_using_gini(X_train, X_test, Y_train) 
clf_entropy = train_using_entropy(X_train, X_test, Y_train) 

# Operational Phase 
print("Results Using Gini Index:") 

# Prediction using gini 
y_pred_gini = prediction(X_test, clf_gini) 
cal_accuracy(Y_test, y_pred_gini) 

print("Results Using Entropy:") 
# Prediction using entropy 
y_pred_entropy = prediction(X_test, clf_entropy) 
cal_accuracy(Y_test, y_pred_entropy) 

#Print tree
feature_names = ['Pulse','BMI','Smoking','Ethnicity','PreviousGDM','Age >30','High Risk','HBa1C',
                     'Creatinine','U Protein', 'U Creatinine']
target_names = ['GDM-Yes','GDM-No']
    
#Build the tree
dot_data = tree.export_graphviz(clf_gini, out_file=None,
                                    feature_names=feature_names,
                                    class_names=target_names, filled = True)
#Draw tree
tr = graphviz.Source(dot_data, format ="png")
tr
  
# tr1 = printTree(clf_gini)
# print("Gini Tree")

# tr2 = printTree(clf_entropy)
# print("Entropy Tree")


# ****Bagging Classifier****

In [ ]:
from sklearn.ensemble import BaggingClassifier

In [ ]:
bag_clf=BaggingClassifier(base_estimator= clf_gini, n_estimators=100, bootstrap=True, n_jobs=-1, random_state=42)
bag_clf.fit(X_train,Y_train)
print ("Accuracy : ", accuracy_score(Y_test,bag_clf.predict(X_test)))